## Logistic Regression - Home Stats

In [19]:
# import required package for data handling
import pandas as pd
import numpy as np

# import required packages for splitting data
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# import required packages for evaluating models
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# import `logistic regression` model
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

In [20]:
# read in the csv
df = pd.read_csv('../csv_files/Home_Only_Classification.csv',index_col=0)
df.head(2)

,H_FTPct,H_EFGPct,H_ThreePARt,H_FTR,H_REBPct,H_BLKPct,H_AST_TOV_Ratio,Target
0,0.833,0.461538,0.395604,0.329670,0.506173,0.036585,1.785714,0
1,0.885,0.430851,0.404255,0.276596,0.538462,0.024390,1.133333,0


In [21]:
# Splitting up our data into variable and target data
X = df.iloc[:, :-1] # Variable
Y = df.Target # Target

In [22]:
# Split dataset into training set and test set
# 70% training and 30% test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=2019)

In [23]:
# initiate the logistic regression function 
logreg = LogisticRegression(solver='liblinear')

# pass the training data into the model. This training data includes all of the independent variables 
logreg = logreg.fit(X_train,Y_train) 

# tell the model what to predict, or in this case classify, and what variables to use to predict the dependent variable 
Y_pred=logreg.predict(X_test)

In [24]:
X.columns

Index(['H_FTPct', 'H_EFGPct', 'H_ThreePARt', 'H_FTR', 'H_REBPct', 'H_BLKPct',
       'H_AST_TOV_Ratio'],
      dtype='object')

In [25]:
array = logreg.coef_
arrayT = array.T

In [26]:
coef = pd.DataFrame(X.columns)
coef['coef'] = arrayT
coef

,0,coef
0,H_FTPct,1.199364
1,H_EFGPct,11.619934
2,H_ThreePARt,-1.645118
3,H_FTR,3.165706
4,H_REBPct,10.105139
5,H_BLKPct,6.732565
6,H_AST_TOV_Ratio,0.715015


In [27]:
coef.sort_values('coef')

,0,coef
2,H_ThreePARt,-1.645118
6,H_AST_TOV_Ratio,0.715015
0,H_FTPct,1.199364
3,H_FTR,3.165706
5,H_BLKPct,6.732565
4,H_REBPct,10.105139
1,H_EFGPct,11.619934


In [28]:
print(array.T)

[[ 1.19936417]
 [11.61993425]
 [-1.64511822]
 [ 3.16570638]
 [10.10513859]
 [ 6.73256475]
 [ 0.71501497]]


In [29]:
logreg.coef_

array([[ 1.19936417, 11.61993425, -1.64511822,  3.16570638, 10.10513859,
         6.73256475,  0.71501497]])

In [30]:
# calculate accuracy, precision and recall? measures of the model 
print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))
print("Precision:",metrics.precision_score(Y_test, Y_pred))
print('Recall:',metrics.recall_score(Y_test, Y_pred))

Accuracy: 0.7751178817555314
Precision: 0.7752747252747253
Recall: 0.8699136868064118


In [31]:
# Calculating accuracy, F1 score, and AUC 
print('Accuracy:', metrics.accuracy_score(Y_test, Y_pred))
print('F1_Score:', metrics.f1_score(Y_test, Y_pred, average='weighted'))
print('AUC:', metrics.roc_auc_score(Y_test, Y_pred, average='weighted'))

Accuracy: 0.7751178817555314
F1_Score: 0.7708411691365019
AUC: 0.7547806319494612


In [32]:
# Calculating R^2
print('R^2:', metrics.r2_score(Y_test, Y_pred))

R^2: 0.07150035035877811


## Evaluation of LR

In [8]:
# lists for f1-score and AUC
f1_score_lst = []
auc_lst = []

#loop to calculate f1 and auc scores and present averages after 10 runs
for count in range (1,10):
    
    #10-fold cross validation
    kfold = model_selection.KFold(n_splits=10, random_state=2019)
    scoring = 'accuracy'
    results = model_selection.cross_val_score(logreg, X_train, Y_train, cv=kfold, scoring=scoring)

    #calculate f1-score and AUC
    clf_roc_auc = roc_auc_score(Y_test, Y_pred)
    f1_score_lst.append(precision_recall_fscore_support(Y_test, Y_pred, average='weighted')[2])
    auc_lst.append(clf_roc_auc)


print('F1 {:.4f}; AUC {:.4f} '.format(np.mean(f1_score_lst),np.mean(auc_lst)))

#result=logit_model.fit()
confusion_matrix_y = confusion_matrix(Y_test, Y_pred)


#print(result.summary())
print('Accuracy of classifier on test set: {:.2f}'.format(logreg.score(X_test, Y_test)))

print("10-fold cross validation average accuracy of classifier: %.3f" % (results.mean()))

print('Confusion Matrix for Logistic Regression Classfier:')
print(confusion_matrix_y)

print('Classification Report for Logistic Regression Classfier:')
print(classification_report(Y_test, Y_pred))

F1 0.9050; AUC 0.9006 
Accuracy of classifier on test set: 0.91
10-fold cross validation average accuracy of classifier: 0.916
Confusion Matrix for Logistic Regression Classfier:
[[1085  160]
 [ 120 1589]]
Classification Report for Logistic Regression Classfier:
              precision    recall  f1-score   support

           0       0.90      0.87      0.89      1245
           1       0.91      0.93      0.92      1709

    accuracy                           0.91      2954
   macro avg       0.90      0.90      0.90      2954
weighted avg       0.91      0.91      0.90      2954

